In [0]:
import numpy as np
import pandas as pd

In [0]:
column_names = ['User', 'Movies', 'rating', 'timestamp']
ratings = pd.read_csv('/content/u.data', sep='\t', names=column_names)

In [0]:
ratings.head()

In [0]:
ratings['User2']=ratings['User'].astype('category')
ratings['Movies2']=ratings['Movies'].astype('category')

In [0]:
ratings.head()

In [0]:
ratings.shape

(100000, 6)

In [0]:
users = ratings.User.unique()
articles = ratings.Movies.unique()

In [0]:
len(users)

943

In [0]:
len(articles)

1682

In [0]:
len(ratings['Movies'][:100000].unique())

1682

In [0]:
ratings['User2']=ratings['User'].astype('category')
ratings['Movies2']=ratings['Movies'].astype('category')

In [0]:
userid2idx = {o:i for i,o in enumerate(users)}
articlesid2idx = {o:i for i,o in enumerate(articles)}

In [0]:
idx2userid = {i:o for i,o in enumerate(users)}
idx2articlesid = {i:o for i,o in enumerate(articles)}

In [0]:
print(articlesid2idx[313],articlesid2idx[60],articlesid2idx[61])

719 330 169


In [0]:
ratings['Movies2'] = ratings.Movies.apply(lambda x: articlesid2idx[x])
ratings['User2'] = ratings.User.apply(lambda x: userid2idx[x])

In [0]:
ratings.head()

,User,Movies,rating,timestamp,User2,Movies2
0,196,242,3,881250949,0,0
1,186,302,3,891717742,1,1
2,22,377,1,878887116,2,2
3,244,51,2,880606923,3,3
4,166,346,1,886397596,4,4


In [0]:
ratings['Movies'].unique()

array([ 242,  302,  377, ..., 1637, 1630, 1641])

In [0]:
n_users = ratings.User.nunique()
n_articles = ratings.Movies.nunique()

In [0]:
n_users

943

In [0]:
import keras.backend as K
def rmse(y_true,y_pred):
  score = K.sqrt(K.mean(K.pow(y_true - y_pred, 2)))
  return score

In [0]:
from keras.layers import Input, Embedding, Dense, Dropout, merge, Flatten
from keras.models import Model

In [0]:
def embedding_input(name,n_in,n_out):
  inp = Input(shape=(1,),dtype='int64',name=name)
  return inp, Embedding(n_in,n_out,input_length=1)(inp)

In [0]:
n_factors = 100
u = embedding_input('user_in', n_users, n_factors)
a = embedding_input('article_in', n_articles, n_factors)

In [0]:
n_factors = 100
user_in, u = embedding_input('user_in', n_users, n_factors)
article_in, a = embedding_input('article_in', n_articles, n_factors)

In [0]:
user_in

<tf.Tensor 'user_in_3:0' shape=(?, 1) dtype=int64>

In [0]:
user_in

<tf.Tensor 'user_in_3:0' shape=(?, 1) dtype=int64>

In [0]:
x = merge([u,a],mode='dot')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [0]:
x=Flatten()(x)
x = Dense(500, activation='relu')(x)
x = Dense(1)(x)

In [0]:
from keras import optimizers

In [0]:
model = Model([user_in,article_in],x)
adam = optimizers.Adam(lr=0.01)
model.compile(adam,loss='mse',metrics=[rmse])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_in (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
article_in (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 1, 100)       94300       user_in[0][0]                    
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 1, 100)       168200      article_in[0][0]                 
__________________________________________________________________________________________________
merge_4 (M

In [0]:
model.fit([ratings.User2,ratings.Movies2], ratings.rating, epochs=50,batch_size=128)

Epoch 1/50
100000/100000 [==============================] - 5s 48us/step - loss: 1.3950 - rmse: 1.1580
Epoch 2/50
100000/100000 [==============================] - 4s 45us/step - loss: 0.8838 - rmse: 0.9378
Epoch 3/50
100000/100000 [==============================] - 5s 45us/step - loss: 0.6383 - rmse: 0.7969
Epoch 4/50
100000/100000 [==============================] - 4s 45us/step - loss: 0.4911 - rmse: 0.6986
Epoch 5/50
 27776/100000 [=======>......................] - ETA: 3s - loss: 0.3365 - rmse: 0.5782

100000/100000 [==============================] - 4s 44us/step - loss: 0.3679 - rmse: 0.6046
Epoch 6/50
100000/100000 [==============================] - 4s 44us/step - loss: 0.2788 - rmse: 0.5260
Epoch 7/50
100000/100000 [==============================] - 5s 45us/step - loss: 0.2195 - rmse: 0.4666
Epoch 8/50
100000/100000 [==============================] - 4s 45us/step - loss: 0.1759 - rmse: 0.4173
Epoch 9/50
 44160/100000 [============>.................] - ETA: 2s - loss: 0.1350 - rmse: 0.3658

100000/100000 [==============================] - 4s 44us/step - loss: 0.1462 - rmse: 0.3805
Epoch 10/50
100000/100000 [==============================] - 5s 46us/step - loss: 0.1219 - rmse: 0.3473
Epoch 11/50
100000/100000 [==============================] - 5s 47us/step - loss: 0.1070 - rmse: 0.3250
Epoch 12/50
100000/100000 [==============================] - 5s 45us/step - loss: 0.0934 - rmse: 0.3038
Epoch 13/50
 39296/100000 [==========>...................] - ETA: 2s - loss: 0.0758 - rmse: 0.2735

100000/100000 [==============================] - 4s 44us/step - loss: 0.0826 - rmse: 0.2855
Epoch 14/50
100000/100000 [==============================] - 5s 45us/step - loss: 0.0726 - rmse: 0.2676
Epoch 15/50
100000/100000 [==============================] - 5s 46us/step - loss: 0.0680 - rmse: 0.2585
Epoch 16/50
100000/100000 [==============================] - 5s 45us/step - loss: 0.0632 - rmse: 0.2491
Epoch 17/50
 38528/100000 [==========>...................] - ETA: 2s - loss: 0.0546 - rmse: 0.2310

100000/100000 [==============================] - 5s 47us/step - loss: 0.0590 - rmse: 0.2403
Epoch 18/50
100000/100000 [==============================] - 5s 47us/step - loss: 0.0561 - rmse: 0.2324
Epoch 19/50
100000/100000 [==============================] - 5s 45us/step - loss: 0.0494 - rmse: 0.2195
Epoch 20/50
100000/100000 [==============================] - 5s 45us/step - loss: 0.0468 - rmse: 0.2130
Epoch 21/50
 36864/100000 [==========>...................] - ETA: 2s - loss: 0.0381 - rmse: 0.1930

...

100000/100000 [==============================] - 5s 46us/step - loss: 0.0240 - rmse: 0.1521
Epoch 34/50
100000/100000 [==============================] - 5s 47us/step - loss: 0.0223 - rmse: 0.1458
Epoch 35/50
100000/100000 [==============================] - 4s 45us/step - loss: 0.0231 - rmse: 0.1463
Epoch 36/50
100000/100000 [==============================] - 4s 45us/step - loss: 0.0256 - rmse: 0.1543
Epoch 37/50
 37504/100000 [==========>...................] - ETA: 3s - loss: 0.0905 - rmse: 0.2204

100000/100000 [==============================] - 5s 46us/step - loss: 0.0520 - rmse: 0.1874
Epoch 38/50
100000/100000 [==============================] - 5s 45us/step - loss: 0.0250 - rmse: 0.1551
Epoch 39/50
100000/100000 [==============================] - 5s 46us/step - loss: 0.0216 - rmse: 0.1370
Epoch 40/50
100000/100000 [==============================] - 4s 44us/step - loss: 0.0171 - rmse: 0.1278
Epoch 41/50
 42368/100000 [===========>..................] - ETA: 2s - loss: 0.0141 - rmse: 0.1158

100000/100000 [==============================] - 4s 45us/step - loss: 0.0159 - rmse: 0.1226
Epoch 42/50
100000/100000 [==============================] - 5s 45us/step - loss: 0.0189 - rmse: 0.1344
Epoch 43/50
100000/100000 [==============================] - 4s 45us/step - loss: 0.0205 - rmse: 0.1400
Epoch 44/50
100000/100000 [==============================] - 5s 46us/step - loss: 0.0209 - rmse: 0.1412
Epoch 45/50
 39552/100000 [==========>...................] - ETA: 2s - loss: 0.0179 - rmse: 0.1308

100000/100000 [==============================] - 5s 46us/step - loss: 0.0174 - rmse: 0.1291
Epoch 46/50
100000/100000 [==============================] - 4s 44us/step - loss: 0.0174 - rmse: 0.1262
Epoch 47/50
100000/100000 [==============================] - 5s 46us/step - loss: 0.0178 - rmse: 0.1288
Epoch 48/50
100000/100000 [==============================] - 4s 44us/step - loss: 0.0156 - rmse: 0.1221
Epoch 49/50
 43520/100000 [============>.................] - ETA: 2s - loss: 0.0158 - rmse: 0.1210

100000/100000 [==============================] - 4s 44us/step - loss: 0.0190 - rmse: 0.1313
Epoch 50/50
100000/100000 [==============================] - 5s 46us/step - loss: 0.0268 - rmse: 0.1492


In [0]:
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
articlesid2idx[59]

574

In [0]:
cosine_similarity(model.get_weights()[1][350].reshape(1,-1),model.get_weights()[1][231].reshape(1,-1))

array([[0.20790277]], dtype=float32)

In [0]:
np.argmax(cosine_similarity(model.get_weights()[1][574].reshape(1,-1),model.get_weights()[1][:574].reshape(574,100)))

169

In [0]:
idx2articlesid[169]

61